In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [64]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math
import numpy as np
import matplotlib.pyplot as plt

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

def randomBasis():
    circuit = QuantumCircuit(1, 1)
    
    U = np.array([[1/np.sqrt(3), -np.sqrt(2)/np.sqrt(3)], 
                  [np.sqrt(2)/np.sqrt(3), 1/np.sqrt(3)]])
    
    circuit.unitary(U, [0], label="U")
    circuit.measure(0, 0)

    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)

    outcome = int(list(counts.keys())[0])

    if outcome == 1:
        circuit = QuantumCircuit(1, 1)
        circuit.h(0)
        circuit.measure(0, 0)

        backend = BasicSimulator()
        compiled = transpile(circuit, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(compiled)
        second_outcome = int(list(counts.keys())[0])

        return 1 if second_outcome == 0 else 2
    else:
        return 0


def average(resultList):
    n = len(resultList)

    count00 = 0
    count01 = 0
    count10 = 0 
    count11 = 0

    for result in resultList:
        if result == "00":
            count00 += 1
        elif result == "01":
            count01 += 1
        elif result == "10":
            count10 += 1
        elif result == "11":
            count11 += 1
    
    return (count00 - count01 - count10 + count11) / n
    

def calculateS(results):
    X_W_avg = average(results["00"])
    X_V_avg = average(results["02"])
    Z_W_avg = average(results["20"])
    Z_V_avg = average(results["22"])
    
    return abs(X_W_avg - X_V_avg + Z_W_avg + Z_V_avg) 
    
N = 20
num_of_steps = (9 * N)//2

aliceBases = [randomBasis() for i in range(num_of_steps)]
print("Alice bases: %s" % aliceBases) 

bobBases = [randomBasis() for i in range(num_of_steps)]
print("Bob bases:   %s" % bobBases) 

root2 = math.sqrt(2)
denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

W_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]

V_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]


aliceKey = []
bobKey = []

resultCounts = {"00":[], "02": [], "20": [], "22":[]}

for i in range(num_of_steps):
    q = QuantumCircuit(2)

    q.h(0)
    q.cx(0,1)
    q.z(1)
    q.x(1)

    if aliceBases[i] == 0:
        q.h(0)
    elif aliceBases[i] == 1:
        q.unitary(W_transform_matrix,[0])

    if bobBases[i] == 0:
        q.unitary(W_transform_matrix,[1])
    elif bobBases[i] == 2:
        q.unitary(V_transform_matrix,[0])
    
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    result = next(iter(counts))
    alice_bit, bob_bit = map(int, result)

    keyScenario = (aliceBases[i] == 1 and bobBases[i] == 0) or (aliceBases[i] == 2 and bobBases[i] == 1)
    calculateSScenario = (aliceBases[i] == 0 and bobBases[i] == 0) or (aliceBases[i] == 0 and bobBases[i] == 2) or (aliceBases[i] == 2 and bobBases[i] == 0) or (aliceBases[i] == 2 and bobBases[i] == 2)

    if keyScenario:
        aliceKey.append(alice_bit)
        bobKey.append(bob_bit)
    elif calculateSScenario:
        key = str(aliceBases[i])+str(bobBases[i])
        resultCounts[key].append(result)

entanglementTestResult = calculateS(resultCounts)

attackerDetected = entanglementTestResult < 2

print("Alice Key: %s" % aliceKey)
print("Bob Key:   %s" % bobKey)
print("Key Length: %d" % len(aliceKey))
print("\nS Counts:   %s" % resultCounts)
print("Entanglement Test Results: %f" % entanglementTestResult)
print("Attacker present?: %s" % attackerDetected)




Alice bases: [1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 1, 2, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 2, 1, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 2, 2, 0, 2, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 1, 0, 1, 0, 2, 2, 1, 0]
Bob bases:   [2, 1, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 0, 1, 1, 1, 0, 2, 1, 0, 2, 0, 0, 0, 0, 1, 2, 2, 1, 1, 0, 1, 1, 2, 1, 2, 0, 0, 1, 0, 1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2, 1, 0, 2, 1, 0, 0, 2, 2, 2, 1, 0, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 2, 0, 2, 0, 1]
Alice Key: [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]
Bob Key:   [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0]
Key Length: 17

S Counts:   {'00': ['00', '11', '11', '00', '00', '00', '00', '11', '11'], '02': ['01', '10', '10', '10', '01', '01', '10', '10', '10', '01', '10'], '20': ['10', '11', '01', '11', '11', '00', '00', '00', '00', '00', '00', '00'], '22': ['11', '11', '10', '00', '00', '11', '11', '00', '00', 